In [ ]:
#Description: The purpose of this report is to build a sentiment anlaysis for three specific stocks, Zack Morris, Will Meade, PJ Matlock, Gary, unusual_whales

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
import numpy as np
import re
from textblob import TextBlob
import textblob
import tweepy
import datetime, time
import sys
import threading

import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt4
from alpaca_trade_api.rest import REST
%matplotlib inline
#import alpaca_trade_api as api

In [ ]:

# authentication and connection details
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = 'https://paper-api.alpaca.markets'

# instantiate REST API
alpaca_api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, base_url, api_version='v2')

# obtain account information
account = alpaca_api.get_account()
print(account)

In [ ]:
load_dotenv()

In [ ]:
consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
Access_Token = os.getenv("Access_Token")
Access_Token_Secret = os.getenv("Access_Token_Secret")

In [ ]:
auth_handler = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth_handler.set_access_token(Access_Token, Access_Token_Secret)
tweepy_api = tweepy.API(auth_handler, wait_on_rate_limit=True)


### Dictionary for Twitter

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'display_text_range', 'entities', 'favorite', 'favorite_count', 'favorited', 'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'possibly_sensitive', 'retweet', 'retweet_count', 'retweeted', 'retweets', 'source', 'source_url', 'truncated', 'user']

TweepError: Twitter error response: status code = 404 (means that the ID does not exist)

In [ ]:
#Variables to build the Tweepy Cursor
tweet_amount = 100
tweets = []
likes = []
time = []


records = []
for user in users:
    for tweet in tweets:
        record['user'] = user
        record['tweet'] = tweet
        record['datetime'] = datetime
        records.append(record)
        

In [ ]:
def clean_tweet(tweetz):
    tweetz = tweetz.replace('@', '')
    tweetz = tweetz.replace('http\S+', '')
    return tweetz

In [ ]:
# Loop function to provide the information for the tweets and Like count

records = []
user_id = ['PJ_Matlock','notoriousalerts','Ultra_Calls']
for user in user_id:
    print(user)
    for i in tweepy.Cursor(tweepy_api.user_timeline, screen_name=user, include_rts=False, tweet_mode='extended').items(tweet_amount):
        record = {}
        record['user'] = user
        record['Tweets'] = clean_tweet(i.full_text)
        record['favorite_count'] = i.favorite_count
        record['created_at'] = i.created_at
        records.append(record)

In [ ]:
tweets_df = pd.DataFrame(records)

In [ ]:
def find_ticker(word):
    if any(ch.isdigit() for ch in word):
        return None
    else:
        return word

In [ ]:
tweets_df['Ticker'] = tweets_df['Tweets'].apply(lambda x: [find_ticker(word) for word in x.split() if '$' in word])

In [ ]:
tweets_df['Polarity'] = tweets_df['Tweets'].map(lambda tweet: textblob.TextBlob(tweet).sentiment.polarity)
tweets_df['Result'] = tweets_df['Polarity'].map(lambda pol: '+' if pol > 0 else '-')
positive = tweets_df[tweets_df.Result == '+'].count()['Tweets']
negative = tweets_df[tweets_df.Result == '-'].count()['Tweets']


In [ ]:
tweets_df

In [ ]:
tweets_df['buy'] = tweets_df['Polarity'] >= .3
tweets_df

In [ ]:
def clean_ticker(row):
    tickers = [ticker for ticker in row if ticker != None]
    return tickers


In [ ]:
tweets_df['buy_list'] = tweets_df['Ticker'].apply(lambda row: clean_ticker(row))
tweets_df

In [ ]:
def buy_ticker(ticker):
    print("using alpacha....")
    symbol_bars = alpaca_api.get_barset(ticker, 'minute', 1).df.iloc[0]
    symbol_price = symbol_bars[ticker]['close']
    alpaca_api.submit_order(
    symbol=ticker,
    qty=50,
    side='buy',
    type='market',
    time_in_force='gtc',
    order_class='bracket',
    stop_loss = {
        'stop_price': symbol_price * 0.95,
        'limit_price':  symbol_price * 0.95
    },
    take_profit={'limit_price': (symbol_price) * 1.10}
    )
    print(f"Just bought {ticker}...")

In [ ]:
for _, tweet in tweets_df.iterrows():
    if tweet['buy'] == True:
        for ticker in tweet['buy_list']:
            ds = tweet['created_at']
            user = tweet['user']
            print(f"buy {ticker} at {ds} based on signal from {user}")
            buy_ticker(ticker[1:])
            #try:
                